<a id="toc"></a>
# HERMES beamline simulation with PyOptiX
***

Contents :
1. [Optical elements declaration](#def_opt)
1. [Definition of optical parameters](#def_param)
1. [Alignment scripts](#alignement)
1. [Simulation execution](#exec)
1. [Visualisation](#visu)

In [1]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '23/JUL/2024'
__changed__ = '25/JUL/2024'

import ctypes

import numpy as np
import pandas as pd
import pyoptix
import pyoptix.classes as px
from pyoptix.ui_objects import plot_spd_plotly
from scipy.constants import c, degree, eV, h, micro, milli, nano, pi, pico
from typing import List

hc = h*c/eV

pyoptix.set_aperture_active(False)
pyoptix.output_notebook()
# benchmarking tools
# %load_ext autoreload
# %autoreload 2
# %matplotlib widget

OptiX library initialized


Loading BokehJS ...

<a id="def_opt"></a>
## Optical elements declaration
[Back to the top](#toc)

In [2]:
# beamline class
Hermes = px.Beamline(name="Hermes - current")

# ------------------
# source
# ------------------
ondulator_LE = px.UndulatorSource(name="ond_LE")         # low energy 
ondulator_HE = px.UndulatorSource(name="ond_HE")         # high energy

# ------------------
# optical elements
# ------------------
pupil = px.PlaneFilm(name="pupil")

# M1
m1a = px.PlaneMirror(name="M1A")
m1b = px.ToroidalMirror(name="M1B")
m1c = px.ToroidalMirror(name="M1C")

# monochromator
mono_entrance_slit = px.PlaneFilm(name="foc_hor")
grating_450 = px.PlaneHoloGrating(name="grating_450")
grating_600 = px.PlaneHoloGrating(name="grating_600")
m2 = px.PlaneMirror(name="M2")
m3 = px.ToroidalMirror(name="M3")
mono_exit_slit = px.PlaneFilm(name="foc_ver")

# branch STXM
m4 = px.ToroidalMirror(name="M4")
vs_m4 = px.PlaneFilm(name="virtual_source_M4")

# branch PEEM
m5 = px.CylindricalMirror(name="M5")
vs_m5 = px.PlaneFilm(name="virtual_source_M5")
m6 = px.ConicCylindricalMirror(name="M6")
m7 = px.ConicCylindricalMirror(name="M7")

# ------------------
# endstations
# ------------------
peem = px.PlaneFilm(name="PEEM")
stxm = px.PlaneFilm(name="STXM")

In [3]:
branches = [[m5, vs_m5, m6, m7, peem],
            [m4, vs_m4, stxm]]

for undulator, m1bis, grating in [(ondulator_LE, m1b, grating_450), (ondulator_HE, m1c, grating_600)]:
    for branch in branches:
        chain_name = f"{undulator.name.split('_')[1]}_G{grating.name.split('_')[1]}_{branch[-1].name}"
        Hermes.chains[chain_name] = [
            undulator, pupil, m1a, m1bis, mono_entrance_slit, grating, m2, m3,
            mono_exit_slit
        ] + branch

for chain_name in Hermes.chains:
    print(chain_name,":\n\t",Hermes.chains[chain_name])

LE_G450_PEEM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> M6 -> M7 -> PEEM 
LE_G450_STXM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 
HE_G600_PEEM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> M6 -> M7 -> PEEM 
HE_G600_STXM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 


<a id="def_param"></a>
## Definition of optical parameters

In this section we define the static parameters of the optical elements. Characteristics that change with 
either configuration or energy are defined in the [Alignment scripts](#alignement) section.

[Back to the top](#toc)

In [4]:
number_of_rays = 5000

### Low energy undulator (**HU64**)

In [5]:
e_beam = px.ElectronBeam()
# e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
#                   emittance=3.94*nano, coupling=1/100,
#                   beta_x=4.7890, eta_x= 0.1804, etap_x= 0.0007, alpha_x=-0.3858,
#                   beta_y=3.7497, eta_y=-0.0044, etap_y=-0.0025, alpha_y=-0.7746)
e_beam.from_rms(energy=2.75,energy_spread=0.1025/100, current=500e-3, x=169e-6, y=6e-6, xp=27e-6, yp=5e-6)
e_beam.print_rms()

hu64 = px.MagneticStructure(period_length=64e-3, number_of_periods=28)

ondulator_LE.electron_beam = e_beam
ondulator_LE.magnetic_structure = hu64

ondulator_LE.write_syned_config(r".\resources\soleil_hu64","Soleil - HU64")

electron beam:
            >> x/xp = 169.00 um vs. 27.00 urad
            >> y/yp = 6.00 um vs. 5.00 urad


### High energy undulator (**HU42**)

In [6]:
e_beam = px.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=4.1825, eta_x=0.1792, etap_x= 0.0007, alpha_x=0.0578,
                  beta_y=2.3439, eta_y=0.0001, etap_y=-0.0025, alpha_y=0.0143)
e_beam.print_rms()

hu42 = px.MagneticStructure(period_length=42e-3, number_of_periods=42)

ondulator_HE.electron_beam = e_beam
ondulator_HE.magnetic_structure = hu42

ondulator_HE.write_syned_config(r".\resources\soleil_hu42","Soleil - HU42")

electron beam:
            >> x/xp = 223.73 um vs. 30.60 urad
            >> y/yp = 9.56 um vs. 4.82 urad


### Entrance pupil
[Back to the top](#toc)

In [7]:
pupil.distance_from_previous = 20
pupil.recording_mode = px.RecordingMode.recording_output
pupil.next = m1a

### M1

[Back to the top](#toc)

In [8]:
m1_grazing_angle = [2.5*degree, 1.2*degree]

m1_distance = px.M1_triad_distances(incident_angle=m1_grazing_angle,
                                    transverse_distance=41e-3, 
                                    verbose=1)

Distance from M1B to M1A: 470.422 mm
- proj. dist. into the optical axis: 468.632 mm

Distance from M1C to M1A: 979.089 mm
- proj. dist. into the optical axis: 978.230 mm



#### - M1A

In [9]:
m1a.distance_from_previous = 0
m1a.phi = -90*degree # rad
m1a.recording_mode = px.RecordingMode.recording_output

#### - M1B

In [10]:
m1b.distance_from_previous = m1_distance[0][0]
m1b.theta = m1_grazing_angle[0]
m1b.phi = 180*degree   
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.recording_mode = px.RecordingMode.recording_output
m1b.next = mono_entrance_slit

#### - M1C

In [11]:
m1c.distance_from_previous = m1_distance[1][0]
m1c.theta = m1_grazing_angle[1] 
m1c.phi = 180*degree   
m1c.minor_curvature = 1/0.8870043 # m-1
m1c.major_curvature = 1/229.5203 # m-1
m1c.recording_mode = px.RecordingMode.recording_output
m1c.next = mono_entrance_slit

### Monochromator

[Back to the top](#toc)

#### Mono entrance slit

In [12]:
mono_entrance_slit.distance_from_previous = 2.7
mono_entrance_slit.phi = -90*degree
mono_entrance_slit.recording_mode = px.RecordingMode.recording_output

#### Grating 450 l/mm

In [13]:
grating_450.distance_from_previous = 0.6
grating_450.line_density = 450/milli
grating_450.inverse_distance1=-0.1761099
grating_450.inverse_distance2=-0.1111111
grating_450.elevation_angle1=-min(np.arccos(0.7986355-0.157995),np.arccos(0.7986355))
grating_450.recording_wavelength=351.1*nano
grating_450.show_vls_law(100e-3, 3)
grating_450.order_align = 1
grating_450.order_use = 1
grating_450.recording_mode = px.RecordingMode.recording_output
grating_450.next = m2

C hologram info for grating_450: 
	 groove/m = 450000.0094938453 + -4913.516826031962 * x + -18413.013775072992 * x^2 + 4956.014995574951 * x^3
	 line radius curvature = 2.4307455744618998
	 line tilt = 0.0


#### Grating 600 l/mm

In [14]:
grating_600.distance_from_previous = 0.6
grating_600.line_density = 600/milli
grating_600.inverse_distance1=-0.2002547
grating_600.inverse_distance2=-0.1111111
grating_600.elevation_angle1=-min(np.arccos(0.7986355), np.arccos(0.7986355-0.21066))
grating_600.recording_wavelength=351.1*nano
grating_600.show_vls_law(100e-3, 3)
grating_600.order_align = 1
grating_600.order_use = 1
grating_600.recording_mode = px.RecordingMode.recording_output
grating_600.next = m2

C hologram info for grating_600: 
	 groove/m = 600000.0163698791 + -483.5964154418325 * x + -29279.683600246906 * x^2 + 8137.222845494747 * x^3
	 line radius curvature = 2.363162423410606
	 line tilt = 0.0


#### M2

In [15]:
m2.phi = 180*degree 
m2.recording_mode = px.RecordingMode.recording_output
m2.next = m3

#### M3

In [16]:
m3.theta = 1.2*degree
m3.phi = -90*degree   
m3.minor_curvature = 1/0.1462124
m3.major_curvature = 1/83
m3.recording_mode = px.RecordingMode.recording_output
m3.next = mono_exit_slit

#### Mono exit sit

In [17]:
mono_exit_slit.distance_from_previous = 3.5
mono_exit_slit.phi = -90*degree
mono_exit_slit.recording_mode = px.RecordingMode.recording_output

### STXM branch

[Back to the top](#toc)

#### M4

In [18]:
m4.distance_from_previous = 2
m4.theta = 1.35*degree
m4.phi = -90*degree
m4.minor_curvature = 1/0.0396
m4.major_curvature = 1/79
m4.recording_mode = px.RecordingMode.recording_output
m4.next = vs_m4

#### virtual source (M4)

In [19]:
vs_m4.distance_from_previous = 1.45
vs_m4.phi = 90*degree
vs_m4.recording_mode = px.RecordingMode.recording_output
vs_m4.next = stxm

#### entrance of the microscope (just before the zone-plate)

In [20]:
stxm.distance_from_previous = 3.5
stxm.recording_mode = px.RecordingMode.recording_output

### PEEM branch

[Back to the top](#toc)

#### M5

In [21]:
m5.distance_from_previous = 1
m5.theta = 1.75*degree
m5.phi = 90*degree
m5.curvature = 1/35
m5.axis_angle = 0 
m5.recording_mode = px.RecordingMode.recording_output
m5.next = vs_m5

#### virtual source (M5)

In [22]:
vs_m5.distance_from_previous = 0.6187585
vs_m5.phi = -90*degree
vs_m5.recording_mode = px.RecordingMode.recording_output
vs_m5.next = m6

#### M6 (KB - HFM)

In [23]:
m6.distance_from_previous = 0.6+7.781242
m6.theta = 1.75*degree
m6.phi = -90*degree
m6.inverse_p = -1/(m6.distance_from_previous)
m6.inverse_q = 1/(0.5 + 1.80)
m6.recording_mode = px.RecordingMode.recording_output
m6.next = m7

#### M7 (KB - VFM)

In [24]:
m7.distance_from_previous = 0.5
m7.theta = 1.75*degree
m7.phi = 90*degree
m7.inverse_p = -1/(m7.distance_from_previous+m6.distance_from_previous+vs_m5.distance_from_previous+m5.distance_from_previous)
m7.inverse_q = 1/(1.80)
m7.recording_mode = px.RecordingMode.recording_output
m7.next = peem

#### PEEM

In [25]:
peem.distance_from_previous = 1.8
peem.recording_mode = px.RecordingMode.recording_output

<a id="alignement"></a>
## Alignment scripts

In this section we define the all the parameters that change with either configuration or energy. These scripts are taylored
for each 

[Back to the top](#toc)

### Undulators

In [26]:
def align_undulator(active_chain, wavelength, **kwargs):
    """
    Sets photon beam size and divergence as a function of wavelength in [m]
    """

    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning undulator")

    active_chain[0].set_undulator(wavelength, **kwargs)
    active_chain[0].waist_x = 200.9266-199.0776
    active_chain[0].waist_y = 200.9266-199.0776
    active_chain[0].sigma_x = 1.733e-04
    active_chain[0].sigma_y = 1.264e-05
    active_chain[0].sigma_x_div = 4.115e-05
    active_chain[0].sigma_y_div = 3.145e-05

### M1 triad

In [27]:
def align_m1(active_chain, m1_distances, **kwargs):
    """
    Sets the M1A angle and distance from mono entrance slit to M1bis
    """
    
    dist_slit = kwargs.get("dist_slit", 2.7)
    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning M1")
   
    active_chain[2].theta = active_chain[3].theta
    active_chain[2].next  = active_chain[3]

    active_chain[4].distance_from_previous = dist_slit
    if "1B" in active_chain[3].name:
        active_chain[4].distance_from_previous += (m1_distances[1][1]-m1_distances[0][1])

    if verbose:
        name = active_chain[3].name
        angle = active_chain[3].theta/degree
        dist = active_chain[3].distance_from_previous
        print(f"M1A-{name} alignment: grazing angle {angle:.2f} degrees and {dist:.3f} m between them")

### Monochromator

In [28]:
def align_mono(active_chain, wavelength, alignment_condition, alignment_condition_value,
               GM2_trans_dist, GM3_proj_dist, **kwargs):
    """
    Sets the grating and M2 angles and relative distances between G/M2/M3
    """

    verbose = kwargs.get("verbose", False)

    grating = None
    for oe in active_chain:
        if "reseau" in oe.name.lower() or "grating" in oe.name.lower():
            grating = oe

    if grating is None:
        raise ValueError("No grating appears in this beamline configuration")
    
    gdict = px.align_grating(grating, verbose=0, 
                             apply_alignment=True, 
                             return_parameters=True,
                             condition=alignment_condition, 
                             condition_value=alignment_condition_value, 
                             lambda_align=wavelength, 
                             order=grating.order_align,
                             line_density=grating.line_density)
    
    gm2 = GM2_trans_dist/np.sin(gdict["deviation"])
    m2m3 = GM3_proj_dist-(GM2_trans_dist/np.tan(gdict["deviation"]))

    active_chain[6].theta = gdict["deviation"]/2
    active_chain[6].distance_from_previous = gm2

    active_chain[7].distance_from_previous = m2m3

    if verbose:
        print(f"\n>>>> {grating.name} grating alignment for a "+
              f"{alignment_condition} {alignment_condition_value} - wavelength " +
              f"{wavelength/nano:.3f} nm (E={hc/wavelength:.3f} eV)")
        print(f"> alpha {gdict['alpha_deg']:.3f} deg")
        print(f"> beta {gdict['beta_deg']:.3f} deg")
        print(f"> G-M2 distance {gm2:.3f} m")
        print(f"> theta_m2 {(gdict['alpha_deg']+gdict['beta_deg'])/2:.3f} deg")
        print(f"> M2-M3 distance {m2m3:.3f} m")

### Beamline alignment procedure

In [29]:
def align_call(alignment_wvl, emission_wvl, alignment_condition, alignment_condition_value, 
               m1_distances, GM2_trans_dist, GM3_proj_dist, **kwargs):

    align_undulator(Hermes.active_chain, emission_wvl, **kwargs)
    align_m1(Hermes.active_chain, m1_distances, **kwargs)
    align_mono(Hermes.active_chain, alignment_wvl, alignment_condition, 
               alignment_condition_value, GM2_trans_dist, GM3_proj_dist, **kwargs)
Hermes.align_steps = align_call

def bl_align(energy_alignment, alignment_condition, alignment_condition_value, 
               m1_distances=m1_distance, GM2_trans_dist=15e-3, GM3_proj_dist=0.55, **kwargs):

    energy_radiate =  kwargs.get("energy_radiate", energy_alignment)
    dE =  kwargs.get("dE", 0)
    rays = kwargs.get("rays", 500)

    alignment_wvl = hc/energy_alignment
    emission_wvl = hc/energy_radiate

    Hermes.align(alignment_wvl, emission_wvl, 
                 alignment_condition=alignment_condition, 
                 alignment_condition_value=alignment_condition_value, 
                 m1_distances=m1_distances, GM2_trans_dist=GM2_trans_dist,
                 GM3_proj_dist=GM3_proj_dist, **kwargs)

    Hermes.clear_impacts(clear_source=True)
    Hermes.active_chain[0].nrays = rays
    if dE==0:
        Hermes.generate(emission_wvl)
    else:
        for E in np.arange(energy_radiate*(1-dE), energy_radiate*(1+dE)):
            Hermes.generate(hc/E)
    Hermes.radiate()

<a id="exec"></a>
## Simulation execution
[Back to the top](#toc)

### Available beamline configurations

In [30]:
print("Low energy configurations:\n")
for chain_name in Hermes.chains:    
    if "450" in chain_name:
        print(chain_name,":\n\t",Hermes.chains[chain_name])

Low energy configurations:

LE_G450_PEEM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> M6 -> M7 -> PEEM 
LE_G450_STXM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 


In [31]:
print("High energy configurations:\n")
for chain_name in Hermes.chains:    
    if "600" in chain_name:
        print(chain_name,":\n\t",Hermes.chains[chain_name])

High energy configurations:

HE_G600_PEEM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> M6 -> M7 -> PEEM 
HE_G600_STXM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 


The beamline can be conviniently simulated by selecting an **active chain** and an energy:

```python
Hermes.active_chain = "LE_G450_STXM"
bl_align(500, alignment_condition="cff", alignment_condition_value=0.2, verbose=True)
```

To check the orientation of the optical elements, draw the beamline:

```python
Hermes.show_active_chain_orientation()
Hermes.draw_active_chain()
spots = Hermes.draw_to_scale()
```

<a id="visu"></a>
## Visualisations 
[Back to the top](#toc)

### HU64 with 450 l/mm grating

In [32]:
Hermes.active_chain = "LE_G450_STXM"
bl_align(500, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=5000, center_undulator=(200.9266-199.0776))

Chaîne LE_G450_STXM:
	ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 

>>>> Aligning undulator
electron beam:
            >> x/xp = 169.00 um vs. 27.00 urad
            >> y/yp = 6.00 um vs. 5.00 urad
filament photon beam:
            >> u/up = 14.53 um vs. 25.67 urad
photon beam waist positon:
            >> hor. x ver. waist position = 0.878 m vs. 1.781 m
convolved photon beam:
            >> x/xp = 169.62 um vs. 37.25 urad
            >> y/yp = 15.72 um vs. 26.15 urad

>>>> Aligning M1
M1A-M1B alignment: grazing angle 2.50 degrees and 0.470 m between them

>>>> grating_450 grating alignment for a cff 0.2 - wavelength 2.480 nm (E=500.000 eV)
> alpha 2.763 deg
> beta 0.552 deg
> G-M2 distance 0.259 m
> theta_m2 1.658 deg
> M2-M3 distance 0.291 m


#### Mono entrance slit

In [36]:
# oe = mono_exit_slit
oe = vs_m4
f = px.get_optimal_focalization_distance(oe, verbose=True)

Optimal focalization along X at -0.369 m from this plane
Optimal focalization along Y at -0.085 m from this plane


In [37]:
px.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=True, save_in_file="", return_fwhm=False,) 

In [38]:
px.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=True, save_in_file="", return_fwhm=False,) 

In [39]:
pupil.distance_from_previous

20.0

In [ ]:
# oe = mono_exit_slit
# px.get_optimal_focalization_distance(oe, verbose=True)
# px.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
#                 show_map=False, light_plot=False, orthonorm=True, save_in_file="", return_fwhm=False,) 